In [ ]:
!pip install pydantic-ai==0.1.3 pandas==2.2.3 | tail -n1

In [1]:
import os
import json
import pandas as pd
from pydantic import BaseModel, validator, Field, field_validator
from openai import OpenAI
from pydantic_ai import Agent, RunContext
import nest_asyncio
from dataclasses import dataclass  

In [6]:
path = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IJIikY8I-Q79KdIA5WtYIw/customer-support-tickets.csv"
df = pd.read_csv(path)

In [7]:
df.head(5)

,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating
0,1,Marisa Obrien,carrollallison@example.com,32,Other,GoPro Hero,2021-03-22,Technical issue,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Social media,2023-06-01 12:15:36,NaN,NaN
1,2,Jessica Rios,clarkeashley@example.com,42,Female,LG Smart TV,2021-05-22,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Chat,2023-06-01 16:45:38,NaN,NaN
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Case maybe show recently my computer follow.,Low,Social media,2023-06-01 11:14:38,2023-06-01 18:05:38,3.0
3,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Try capital clearly never color toward story.,Low,Social media,2023-06-01 07:29:40,2023-06-01 01:57:40,3.0
4,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,West decision evidence bit.,Low,Email,2023-06-01 00:12:42,2023-06-01 19:53:42,1.0


In [8]:
client = OpenAI()

In [9]:
from pydantic import BaseModel, Field, field_validator

# SupportResponse using Pydantic V2

class SupportResponse(BaseModel):
    category: str = Field(..., description="The issue category")
    priority: str = Field(default="low", description="Issue priority (low, medium, high)")
    escalate: bool = Field(..., description="Should this be escalated to a human?")
    suggested_response: str = Field(..., description="A helpful support reply")

    @field_validator("priority", mode="before")
    @classmethod
    def validate_priority(cls, v):
        valid_priorities = {"low", "medium", "high"}
        if isinstance(v, str) and v.lower() in valid_priorities:
            return v.lower()
        return "low"  # fallback if invalid

    def priority_as_int(self) -> int:
        priority_map = {"low": 1, "medium": 2, "high": 3}
        return priority_map.get(self.priority, 1)

In [10]:
prompt = """
User: I forgot my password and can’t login.
Response:
{
  "category": "Authentication",
  "priority": "High",
  "escalate": true,
  "suggested_response": "You can reset your password from the login page. If that doesn’t work, we'll escalate this issue for manual support."
}

User: Please cancel my account.
Response:
{
  "category": "Subscription",
  "priority": "Medium",
  "escalate": false,
  "suggested_response": "To cancel your subscription, go to Settings > Billing > Cancel Plan. Contact us if you need help."
}
"""

In [11]:
agent = Agent[SupportResponse](
    instructions=f"""
You are a helpful customer support assistant.
You can use this CSV file: {df}

Your task is to:
- Match user input to any ticket using ticket_id, email, or customer_name.
- Respond ONLY using the data in the CSV.
- If no match is found, respond politely asking the user to check their input.
- Output MUST be a valid JSON object in this exact format (no markdown, no prose):

{{
  "category": "Connectivity",
  "priority": "high",
  "escalate": true,
  "suggested_response": "Please update your Wi-Fi drivers and restart your router."
}}

Only return the JSON — no extra text or explanation.
""",
    model="gpt-4.1-nano",
    client=client,
)

In [12]:
nest_asyncio.apply()

In [13]:
import json
from pydantic import ValidationError

async def run_chatbot():
    print(" Customer Support Chatbot — type 'exit' to quit\n")

    while True:
        user_input = input("🧑 You: ").strip()
        if user_input.lower() == "exit":
            print(" Exiting chatbot.")
            break

        try:
            result = await agent.run(user_input)
            output = result.output

            # Try parsing the result into SupportResponse
            if isinstance(output, SupportResponse):
                response = output
            elif isinstance(output, dict):
                response = SupportResponse(**output)
            elif isinstance(output, str):
                response = SupportResponse(**json.loads(output))
            else:
                raise ValueError("Unsupported output format from agent.")

            # ✅ Print structured response
            print("\nAI Response:")
            print(f"Category        : {response.category}")
            print(f"Priority        : {response.priority}")
            print(f"Escalate        : {'Yes' if response.escalate else 'No'}")
            print(f"Suggested Reply : {response.suggested_response}\n")

        except (ValidationError, ValueError, json.JSONDecodeError) as e:
            print("\n❌ Could not parse structured output from agent:")
            print(output)

# Run this to launch chatbot
await run_chatbot()

 Customer Support Chatbot — type 'exit' to quit



🧑 You:  Christopher Robbins purchased a Dell XPS. If you need more details, please let us know



AI Response:
Category        : Technical issue
Priority        : medium
Escalate        : No
Suggested Reply : Please ensure your Dell XPS is properly connected and updated with the latest drivers. If the issue persists, contact our support team for further assistance.



🧑 You:  My Dell XPS keeps dropping from Wi-Fi every few minutes, which is really frustrating. I’d appreciate some help troubleshooting this issue



AI Response:
Category        : Connectivity
Priority        : high
Escalate        : Yes
Suggested Reply : Please update your Wi-Fi drivers and restart your router.



🧑 You:  exit


 Exiting chatbot.


dish_name='Veggie Stir-fry with Quinoa' description="A quick and energizing dish that's loaded with colorful vegetables and protein-rich quinoa to boost your energy levels." ingredients=['1 cup quinoa', '2 cups water', '1 tablespoon olive oil', '2 cloves garlic, minced', '1-inch piece ginger, minced', '1 red bell pepper, sliced', '1 yellow bell pepper, sliced', '1 cup broccoli florets', '1 cup snap peas', '2 carrots, julienned', '2 tablespoons soy sauce', '1 tablespoon sesame oil', '1 tablespoon lime juice', 'Salt and pepper to taste', 'Fresh cilantro for garnish'] cooking_time_minutes=30


/tmp/ipykernel_820/997800736.py:49: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  print(result.data)
